# 7. Наивный Байес и Классификация текстов

> Байесовские алгоритмы для классификации текстов  
> Извлечение признаков из текста

1. Скачайте этот ноутбук к себе.
2. Заполните пропущенные ячейки, отвечая на заданные вопросы. Там должен быть код! (если не сказано обратное)
3. Сохраните результат в своём гитхаб репозитории.

### Полезная литература

- [Naive Bayes and Text Classification](https://sebastianraschka.com/Articles/2014_naive_bayes_1.html)
- [ODS - классификация](https://habrahabr.ru/company/ods/blog/322534/)
- [Байесовский классификатор](http://www.machinelearning.ru/wiki/index.php?title=%D0%91%D0%B0%D0%B9%D0%B5%D1%81%D0%BE%D0%B2%D1%81%D0%BA%D0%B8%D0%B9_%D0%BA%D0%BB%D0%B0%D1%81%D1%81%D0%B8%D1%84%D0%B8%D0%BA%D0%B0%D1%82%D0%BE%D1%80)
- [Интересная статья про спам/не спам](https://habrahabr.ru/post/252265/)

Если вам понравилась задача распознавания текстов, то вот подборочка:

- [библиотека Gensim](https://radimrehurek.com/gensim/)
- [библиотека nltk](http://www.nltk.org/)
- [Лекция про word2vec 🔥](https://www.youtube.com/watch?v=oBb9aFmp0Hs)
- [Лекция Векторные представления слов и документов 🔥](https://www.youtube.com/watch?v=hiDBnEyoZS4)

## Наивный байесовский классификатор

На практике мы использовали его для детекции спам/не спам. В этот раз вы будете классифицировать [новостные тексты](https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html). Попробуете разные методы  генерации признаков из текстов.

In [3]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

### 2.1 Загрузка данных (2 балла)

Мы будем использовать только 4 класса текстов: `'alt.atheism', 'sci.space', 'talk.religion.misc', 'comp.graphics'`.

Загрузите тренировочную и тестовую части датасета для этих категорий.

In [4]:
classes = ['alt.atheism','sci.space','talk.religion.misc','comp.graphics']
newsgroups_train = fetch_20newsgroups(subset='train', categories=classes, remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', categories=classes, remove=('headers', 'footers', 'quotes'))

Выведите по 1 тексту для каждой из категории.

In [5]:
newsgroups_train.data[0]

"Hi,\n\nI've noticed that if you only save a model (with all your mapping planes\npositioned carefully) to a .3DS file that when you reload it after restarting\n3DS, they are given a default position and orientation.  But if you save\nto a .PRJ file their positions/orientation are preserved.  Does anyone\nknow why this information is not stored in the .3DS file?  Nothing is\nexplicitly said in the manual about saving texture rules in the .PRJ file. \nI'd like to be able to read the texture rule information, does anyone have \nthe format for the .PRJ file?\n\nIs the .CEL file format available from somewhere?\n\nRych"

In [6]:
newsgroups_test.data[0]

'TRry the SKywatch project in  Arizona.'

### 2.2 Мешок слов (6 баллов)

Начнем с самого простого способа извлечения признаков из текстов: обычный подсчет количества употреблений слов в тексте.

Для этого используем [sklearn.feature_extraction.text.CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html). Настройте его так, чтобы он использовал англиские стоп слова.

Проведите трансформаци текстов из обоих датасетов и выведите размер полученного словаря. (1 векторайзер для тестового и тренировочного датасета! У вас должны получиться матрицы с одинаковыми признаками-словами)

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
y_train = newsgroups_train.target
X_train = newsgroups_train.data

y_test = newsgroups_test.target
X_test = newsgroups_test.data

X_share = X_train + X_test;
vectorizer = CountVectorizer(stop_words='english')
X_vector_share = vectorizer.fit_transform(X_share)
X_vector_train = X_vector_share[:len(X_train)]
X_vector_test = X_vector_share[len(X_train):]
print(X_vector_train.shape)
print(X_vector_test.shape)

(2034, 33529)
(1353, 33529)


Будем использовать классифиактор для мультиклассифкации [sklearn.naive_bayes.MultinomialNB]().

- Проведите кроссвалидацию на 5 фолдах для метрики `accuracy`.
- обучите на тренировочных данных, 
- нарисуйте красивую confusion матрицу, (с именами классов, а не индексами по осям)
- выведите отчет классификации для тестовой выборки (с именами классов, а не индексами)

In [16]:
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
res = cross_val_score(MultinomialNB(), X_vector_train, y_train, cv=5)
res.mean(), res.std()

(0.7905661851976931, 0.01727902250014084)

In [9]:
clf = MultinomialNB()
clf.fit(X_vector_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [10]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = clf.predict(X_vector_test)
print(classification_report(y_test, y_pred, target_names = newsgroups_train.target_names))

cm = confusion_matrix(y_test, y_pred)
for i in range(4):
    print("{} {}".format(cm[i], newsgroups_train.target_names[i]))

                    precision    recall  f1-score   support

       alt.atheism       0.65      0.71      0.68       319
     comp.graphics       0.91      0.91      0.91       389
         sci.space       0.84      0.87      0.85       394
talk.religion.misc       0.66      0.54      0.60       251

          accuracy                           0.78      1353
         macro avg       0.76      0.76      0.76      1353
      weighted avg       0.78      0.78      0.78      1353

[227   8  26  58] alt.atheism
[ 12 353  21   3] comp.graphics
[ 25  19 342   8] sci.space
[ 87   8  20 136] talk.religion.misc


In [11]:
import numpy as np
# Посмотрим на ТОП 10 по важности слова в каждой категории

def show_top10(classifier, vectorizer, categories):
    feature_names = np.asarray(vectorizer.get_feature_names())
    for i, category in enumerate(categories):
        top10 = np.argsort(classifier.coef_[i])[-10:]
        print("%s: [%s]" % (category, " ".join(feature_names[top10])))

show_top10(clf, vectorizer, newsgroups_train.target_names)

alt.atheism: [like believe say atheism does just think don people god]
comp.graphics: [software images files data use file jpeg edu graphics image]
sci.space: [just shuttle time orbit data like earth launch nasa space]
talk.religion.misc: [say know christian think just bible don jesus people god]


### 2.3 [Tf-IDF](https://ru.wikipedia.org/wiki/TF-IDF) (6 баллов) 

Мешок слов не учитывает "веса" слов, он просто смотрит их вхождение в документ. Вероятно, было бы полезно взвесить каким-то обраом каждое слово в документе. Действительно, если слово встречается во всех документах, то, наверное, его вес небольшой. А если редкое слово встречается в некоторых документах, то скорее всего оно какое-то узко тематическое.

Один из способов взвесить слова - это использовать меру tf-idf, где:

**TF - term frequency** - частота слова для каждой статьи

$$\LARGE \mathrm{tf}(t,d) = \frac{n_t}{\sum_k n_k}$$

**IDF - inverse document frequency*** — обратная частота документа - уменьшает вес часто встречаемых слов

$$\LARGE \mathrm{idf}(t, D) =  \log \frac{|D|}{|\{\,d_i \in D \mid t \in d_{i}\, \}|}$$

$D$ - число документов в корпусе

$|\{\,d_i \in D \mid t \in d_{i}\, \}|$ - число документов из коллекции ${\displaystyle D}$ , в которых встречается ${\displaystyle t}$  (когда ${\displaystyle n_{t}\neq 0}$ ).

**TF-IDF**

$$\LARGE \operatorname{tf-idf}(t,d,D) = \operatorname{tf}(t,d) \times \operatorname{idf}(t, D)$$


Для него есть класс [sklearn.feature_extraction.text.TfidfVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

- Проведите кроссвалидацию на 5 фолдах для метрики `accuracy`.
- обучите на тренировочных данных, 
- нарисуйте красивую confusion матрицу, (с именами классов, а не индексами по осям)
- выведите отчет классификации для тестовой выборки (с именами классов, а не индексами)

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
y_train = newsgroups_train.target
X_train = newsgroups_train.data

y_test = newsgroups_test.target
X_test = newsgroups_test.data

X_share = X_train + X_test;
vectorizer = TfidfVectorizer(stop_words='english')
X_vector_share = vectorizer.fit_transform(X_share)
X_vector_train = X_vector_share[:len(X_train)]
X_vector_test = X_vector_share[len(X_train):]
print(X_vector_train.shape)
print(X_vector_test.shape)
%time cross_val_score(MultinomialNB(), X_vector_train, y_train, cv=5).mean()

(2034, 33529)
(1353, 33529)
CPU times: user 96.8 ms, sys: 0 ns, total: 96.8 ms
Wall time: 96.6 ms


0.7905661851976931

In [13]:
clf = MultinomialNB()
clf.fit(X_vector_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [14]:
y_pred = clf.predict(X_vector_test)
print(classification_report(y_test, y_pred, target_names = newsgroups_train.target_names))

cm = confusion_matrix(y_test, y_pred)
for i in range(4):
    print("{} {}".format(cm[i], newsgroups_train.target_names[i]))

                    precision    recall  f1-score   support

       alt.atheism       0.60      0.70      0.65       319
     comp.graphics       0.87      0.92      0.90       389
         sci.space       0.73      0.92      0.82       394
talk.religion.misc       0.79      0.22      0.34       251

          accuracy                           0.74      1353
         macro avg       0.75      0.69      0.67      1353
      weighted avg       0.75      0.74      0.71      1353

[224  17  64  14] alt.atheism
[  4 359  26   0] comp.graphics
[  9  21 364   0] sci.space
[136  16  45  54] talk.religion.misc


In [15]:
show_top10(clf, vectorizer, newsgroups_train.target_names)

alt.atheism: [islam does religion atheism say just think don people god]
comp.graphics: [does looking hi program know file image files thanks graphics]
sci.space: [lunar earth think orbit just moon launch like nasa space]
talk.religion.misc: [objective think just don bible christians christian people jesus god]


### 2.4 В чём проблема? (7 баллов)

Как вы заметили, качество немного улучшилось. Как вы думаете почему у нас до сих пор есть ошибки? В чём проблема? Опирайтесь на вывод топа 10 важных слов, confusion matrix и репорт классификации.

**Задание**: Напишите развернутое рассуждение.

Ответ: 

Думаю, основная причина в том, что многие слова встречаются в нескольких группах. Например, слово religion может одинаково часто встречаться как в религиозных текстах, так и в атеистических.

По матрице можно заметить, что лучше всего определяются тексты о космосе и о графике. Видимо, из-за частого употребления слов, не встречающихся в других классах. Половина Топа-10 - это слова уникальные для данных тематик. С атеизмом и религией сложнее. Слова в Топе хоть и разные, но схожие по тематике, вполне вероятно, что там ущё куча одинаковых слов.

Ну и слова does и don't тоже делу не помогают :D

![](https://i.imgur.com/3D5pgrk.png)

Можно еще немного улучшить качество предсказания, если использовть n-граммы, то есть не только одиночные слова, но и пары, тройки. Для этого у векторайзера есть параметр `ngram_range`, но будьте осторожны, сгенерируется очень много признаков. Их количество можно ограничить  `max_features`. Ещё можно копать в сторону правильного процессинга текстов: лемматизация и стемминг, чтобы убирать окончания, но это обчно не сильно помогает в английском языке.

Кроме того можно было бы пойти совсем в другую сторону и использовать word-2-vec технологии, тогда бы у нас слова начал "обретать" смысл. Но это вовсем другая история.